# Sending utterances and generating logs

This notebook demonstrates a technique to programmatically Send utterances and generate conversation logs for a workspace in <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant</a>.

Sometimes you need a quick way to simulate a user and call the /message API with a sequence of input utterances. For example, you may want to test your workspace and generate data for the Improve section of the Conversation UI. you are going  to use this notebook with the sample utterances-for-generate-chat-logs CSV file to do this. 

This notebook will demonstrate how the Watson Assistant API can be directly accessed to programmatically send user examples. This is an alternative to the user GUI.

This notebook runs on Python 3.5
So, on the top right be sure to run the right Kernel, if not go to menu **Kernel > Change Kernel** then select **Python 3.5** 

**Tips:**
* Code cells are identifiable by their `In [ ]:` prefix in the margin
* To execute the celsl in the notebook, select the cell and click the run button, or hit Ctrl-Enter.
* Cells which have not been executed before will have empty brackets, while executed cells will have a sequence number within, e.g. `In [13]`
* Cell execution result displays below the cell
* To clear all exection statuses and outputs, use the `Cell/All Output/Clear` menu.

Then execute the cell (Ctrl-Enter or run button)

## Table of contents

1. [Install and import packages](#setup)
2. [Import the data as a pandas DataFrame](#import)
3. [Authenticate to the Watson Assistant Service](#authenticate)
4. [Test the connection to the Watson Assistant](#wcs1)
5. [Evaluate the test set with the message function](#wcs4)<br>
[Summary and next steps](#Summary-and-next-steps)

## <a id="setup"></a> Step 1. Install and import packages

Install and import the necessary packages.


In [ ]:
!pip install --upgrade watson-developer-cloud

In [ ]:
import pandas as pd
import numpy as np
#from bokeh.charts import Histogram, output_file, show
import random
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split

## <a id="import"></a>Step 2. Import the data as a pandas DataFrame

The data consists of sample user questions and the assigned intents. 

**For notebooks running on IBM Data Science Experience:**

To get the data and load it into a pandas DataFrame:

* Select the code cell below, and **delete all its content**
* Open the data panel on the right using the 1001 button icon  (top right)
* Drop your file "utterances-for-generate-chat-logs.csv" with the your intents and user examples.
* From the data panel on the right use context menu on the added file choose **Insert to code > Insert Pandas DataFrame** 

Some code should be generated, which creates a `df_data_1` panda DataFrame. If the name is different, change the variable name back to `df_data_1`

**For Python notebook servers**
1. Uncomment and modify the code stub to load data from your server's filesystem. 

**File description**
Given a CSV file containing utterances to simulate a user's questions, and a set of input parameters, this script will call the Watson Conversation /message API and simulate an interaction between a user and the Conversation service. The result will include data in the Improve tab of the Watson Conversation Service UI. The CSV file format allows you to include a sequence of utterances in the same conversation. Place each utterance on a separate line. To start a new conversation, you should include a line with the string :init:
For example, a CSV to simulate two conversations with the Car Dashboard Demo sample application # could be as simple as these six lines:
question
hi
turn lights on
:init:
hi
hit the brake

In [ ]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
<Credential>
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



Rename the DataFrame to `test`:

In [ ]:
# Make sure this uses the variable above. The number will vary in the inserted code.
try:
    test = df_data_1
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    print('Follow the instructions to insert the pandas DataFrame above, and edit to')
    print('make the generated df_data_# variable match the variable used here.')
    raise
test.head()

## <a id="authenticate"></a>Step 3. Authenticate to the Watson Assistant

Sign up for the Watson Conversation service and enter your credentials. 

1. Sign up for [Watson Assistant](https://console.bluemix.net/catalog/services/conversation) in IBM Cloud.
1. On your Watson Conversation service page, click **Launch Tool**. The Workspaces page appears in a separate tab.
1. On your Watson Conversation Workspaces page, Select your workspace which simulate your Production environment. 
1. Find your workspace ID and credentials by clicking the **Deploy** button and then **Credentials**. 
1. Add your workspace ID, username, and password to the next cell and run the cell.

Tips:
* The Watson Studio and your Watson services must be in the same IBM Cloud region (US South for instance)

In [ ]:
CONVERSATION_APIKEY = '<IAMAPIKey>'
CONVERSATION_URL = 'https://gateway.watsonplatform.net/assistant/api'
VERSION = '2018-07-10'
WORKSPACE_ID = '<WorkspaceID>'
DEPLOYMENT_ID='ChatbotProduction'
USER_ID='test'
CUSTOMER_ID=''

Import the Watson Assistant package and set variables:

In [ ]:
import json
from watson_developer_cloud import AssistantV1
assistant = AssistantV1(
    version=VERSION,
    iam_apikey=CONVERSATION_APIKEY,
    url=CONVERSATION_URL
)

## <a id="wcs1"></a>Step 4. Test the connection to the Watson Assistant
Run the <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant API</a> functions to make sure you are properly connected to your Watson Assistant Workspace.

List the existing intents with the `list_intents` function. If this is the first time you're using the Watson Assistant, you won't have any intents.

In [ ]:
intents = assistant.list_intents(WORKSPACE_ID).get_result()
print(json.dumps(intents, indent=2))

## <a id="wcs4"></a>Step 5. Evaluate the test set with the message function
    Now send user examples. By using the `message` function from the <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant API</a>, you can test all examples at once, instead of examining each example individually with the Assistant Workspace tool.

In [ ]:
from __future__ import print_function  # Python 3 support

import sys
import datetime
import re
import csv
import argparse
     
if CUSTOMER_ID is not None:
    assistant.set_default_headers({'X-Watson-Metadata': 'customer_id=' + CUSTOMER_ID})

#2. Built the message metadata and context if provided
""" 
Builds the context metadata if deployment or user_id are specified as arguments
"""
deploy_tag = { "deployment": DEPLOYMENT_ID} if DEPLOYMENT_ID is not None else None
user_id = { "user_id": USER_ID} if USER_ID is not None else None
metadata = None
if deploy_tag is not None:
    if user_id is not None:
        metadata = deploy_tag.copy()
        metadata.update(user_id)
    else:
        metadata = deploy_tag
else:
    if user_id is not None:
        metadata = user_id
mycontext = {'metadata': metadata} if metadata is not None else None
    
# 3. Read the CSV file
#  Done previously
test_result = True
turn_count = 0

    # 4. Process the csv file
    
#    for turn in csv_reader:
for turn in [x[1] for x in test[:].iterrows()]:    
    turn_count += 1
    if turn['question'][0] == ';':
        continue
    if turn['question'] == ':init:':
        mycontext = {'metadata': metadata} if metadata is not None else None
        turn_count = 0
        continue

    response = None
    print('Input : %s' % turn['question'])
#    print(json.dumps(mycontext, indent=2))
    try:
        response = assistant.message(workspace_id=WORKSPACE_ID, input={'text': turn["question"]}, context=mycontext).get_result()
    except WatsonApiException as error:
        print(error)
        sys.exit(1)

    mycontext = response['context']
    output = ' '.join(str(x) for x in response['output']['text'])

    if turn_count == 1:
        print('Conversation ID: %s (%s)' % (mycontext['conversation_id'], str(datetime.datetime.now())))
        
    #print('Input : %s' % turn['question'])
    print('Output: %s\n' % output)


## Summary and next steps
You've learned how to use the Watson Assistant API to send user examples. Try adding your own user questions and see how Watson does!
you must review the 'improve' feature of Watson assistant to see what were captured by the service.

Learn more:
- <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant API reference</a>
- <a href="https://github.com/watson-developer-cloud/python-sdk" target="_blank" rel="noopener noreferrer">Watson Assistant Python SDK</a>

### Authors
Laurent Vincent.

Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.